In [101]:
import numpy as np
import pandas as pd
from scipy.misc import derivative
from scipy.optimize import minimize, curve_fit

In [102]:
N = 4
INF = 10 ** 6
I = 1000

x0 = np.zeros(N)
h0 = np.zeros(N)

p = [20, 20, 20, 20] # initial points

# Part I

### My function

$ U(x) = (x_{1}-7)^{0.25}(x_{2}-17)^{0.09}x^{0.3}_{3}(x_{4}-2)^{0.2} $

In [103]:
def U(x1, x2, x3, x4):
    return (x1 - 7) ** 0.25 * (x2 - 17) ** 0.09 * x3 ** 0.3 * (x4 - 2) ** 0.2


def partial_derivative(func, var=0, point=[]):
    args = point[:]

    def wraps(x):
        args[var] = x
        return func(*args)

    return derivative(wraps, point[var], dx = 1e-6)

## Marshall

In [104]:
def objective_m(x):
    return - U(x[0], x[1], x[2], x[3])


def constraint1(x):
    return I - x[0] * p[0] - x[1] * p[1] - x[2] * p[2] - x[3] * p[3]


x0[0] = 9 # initial x-vector
x0[1] = 20
x0[2] = 9
x0[3] = 10

bnds = (
    (7, INF),
    (17, INF),
    (0, INF),
    (2, INF)
)

con1 = {'type': 'ineq', 'fun': constraint1}
cons = ([con1])
solution = minimize(objective_m, x0, method='SLSQP', bounds=bnds, constraints=cons)

x = solution.x

print('Final Objective: ' + str(-objective_m(x)))

print('\nSolution')
for i in range(4):
    print('x{} = {}'.format(i + 1, x[i]))
    
print("\nLagrange multiplier")
L = [partial_derivative(U, i, x) / p[i] for i in range(4)]
for i in range(4):
    print('L{} = {}'.format(i + 1, L[i]))

Final Objective: 4.80498489644

Solution
x1 = 14.140279462615863
x2 = 19.570974676129996
x3 = 8.575006773194632
x4 = 7.713739088059505

Lagrange multiplier
L1 = 0.008411759155890763
L2 = 0.008410208418574427
L3 = 0.008405215323747939
L4 = 0.008409528917674436


## Hiks

In [105]:
def objective_h(h):
    return p[0] * h[0] + p[1] * h[1] + p[2] * h[2] + p[3] * h[3]


def constraint1(h):
    return U(h[0], h[1], h[2], h[3]) - 200


h0[0] = 9 # initial h-vector
h0[1] = 20
h0[2] = 9
h0[3] = 10

bnds = (
    (7, INF),
    (17, INF),
    (0, INF),
    (2, INF)
)

con1 = {'type': 'ineq', 'fun': constraint1}
cons = ([con1])
solution = minimize(objective_h, h0, method='SLSQP', bounds=bnds, constraints=cons)

h = solution.x

print('Final Objective: ' + str(objective_h(h)))

print('\nSolution')
for i in range(len(h)):
    print('h{} = {}'.format(i + 1, h[i]))
print('U(h) = {}'.format(U(h[0], h[1], h[2], h[3])))

print("\nLagrange multiplier")
L = [p[i] / partial_derivative(U, i, h) for i in range(4)]
for i in range(4):
    print('L{} = {}'.format(i + 1, L[i]))

Final Objective: 41166.6268386

Solution
h1 = 611.8566384737018
h2 = 234.74952920994335
h3 = 725.8345065912623
h4 = 485.8906676551041
U(h) = 200.00000000000225

Lagrange multiplier
L1 = 241.94265914293283
L2 = 241.9439901104927
L3 = 241.94486356591216
L4 = 241.94536268612626


# Part II

The volume of production capital (fixed assets) K and the volume of labor L,
which are measured: for K - in monetary or natural terms, and for
L - in the number of workers, man-hours, man-days, etc.

### Uploading data (variant #8)

In [106]:
data = pd.read_csv('lab3_data.csv')
data

,K,L,F
0,2660,10380,49420
1,2720,10210,45250
2,2740,10370,50050
3,2650,10500,50070
4,2320,9750,47320
5,2490,10100,47400
6,2770,10620,51400
7,2610,10380,45470
8,2800,9600,47530
9,2880,9570,47600


In [107]:
F = data['F'].values
K = data['K'].values
L = data['L'].values

## Production func., scales and elasticity

In [112]:
def cobb_douglas(x, a, b, c):
    return a * (x[0] ** b) * (x[1] ** c)


p0 = [1, 1, 1]
coeffs, _ = curve_fit(cobb_douglas, (K, L) , F, p0)

print("Production function K = {} * F ^ {} * L ^ {}".format(coeffs[0], coeffs[1], coeffs[2]))

if abs(coeffs[1] + coeffs[2] - 1) < 1e-3:
    print('Constant returns to scale')
elif coeffs[1] + coeffs[2] > 1:
    print('Increasing returns to scale')
else:
    print('Decreasing returns to scale')

print('Elasticity of substitution for Cobb–Douglas production function is const=1')

Production function K = 141.69191034852955 * F ^ 0.15111129031060572 * L ^ 0.5026447942457296
Decreasing returns to scale
Elasticity of substitution for Cobb–Douglas production function is const=1


## Short-run & long-run profits

In [109]:
price = 70
w = [100, 100]


def pi(x):
    return w[0] * x[0] + w[1] * x[1] - price * cobb_douglas(x, coeffs[0], coeffs[1], coeffs[2])

def constraint1(x):  # sqrt(x[0] * x[0] + x[1] * x[1]) <= 5000
    return 5000 - (x[0] * x[0] + x[1] * x[1]) ** 0.5


bounds1 = (
    (0, INF),
    (0, INF)
)

long_solution = minimize(pi, [1, 1], method='SLSQP', bounds=bounds1, constraints=[])
print('Solution')
x = long_solution.x
for i in range(len(x)):
    print('x{} = {}'.format(i + 1, x[i]))
print("Long run profit is", -pi(long_solution.x))
print()

con1 = {'type': 'ineq', 'fun': constraint1}
short_solution = minimize(pi, [1, 1], method='SLSQP', bounds=bounds1, constraints=[con1])
print('Solution')
x = short_solution.x
for i in range(len(x)):
    print('x{} = {}'.format(i + 1, x[i]))
print("Short run profit is", -pi(short_solution.x))

Solution
x1 = 14240.574014870512
x2 = 47367.09422920981
Long run profit is 3262295.00062

Solution
x1 = 2028.6739748934274
x2 = 4569.954256679813
Short run profit is 1507022.09882


## Monopoly profit/price (of resources) and the volume of production

In [111]:
def price_func(x):
    return -x / 830 + 8310/83


def wL(x):
    return 0.025 * x[1] - 20


def wF(x):
    return 0.025 * x[0] - 10


def wM(x):
    return (wF(x), wL(x))


def monopoly_pi(x):
    q = cobb_douglas(x, coeffs[0], coeffs[1], coeffs[2])
    mw = wM(x)
    return mw[0] * x[0] + mw[1] * x[1] - price_func(q) * q


monopoly_solution = minimize(monopoly_pi, [1, 1], method='SLSQP', bounds=bounds1, constraints=[])
print('Solution')
x = monopoly_solution.x
for i in range(len(x)):
    print('x{} = {}'.format(i + 1, x[i]))
print("Monopoly profit is", -monopoly_pi(monopoly_solution.x))
print("Monopoly price is", price_func(cobb_douglas(monopoly_solution.x, coeffs[0], coeffs[1], coeffs[2])))
print("Monopoly price of resources: F = {}, L = {}".format(wF(monopoly_solution.x), wL(monopoly_solution.x)))
print("Volume of production is", cobb_douglas(monopoly_solution.x, coeffs[0], coeffs[1], coeffs[2]))

Solution
x1 = 1819.053345464246
x2 = 3336.1107202697835

Monopoly profit is 1512419.97127
Monopoly price is 68.8016982418
Monopoly price of resources: F = 35.47633363660615, L = 63.402768006744594
Volume of production is 25994.5904593
